In [ ]:
## IMPORTING THE DATA
import pandas as pd
import numpy as np
df = pd.read_csv("../input/ipl-dataset/all_matches.csv",low_memory = False)

In [ ]:
## PERFORMING EDA AND DATA CLEANING
df = df[df.ball <= 6.0]
df = df[df.innings < 3]

In [ ]:
df = df.fillna(0)

In [ ]:
df['total_runs'] = df.runs_off_bat + df.extras

from datetime import datetime
df['start_date'] = df['start_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
df.drop(['runs_off_bat','season','extras','wides','noballs','byes','legbyes','penalty','other_player_dismissed','other_wicket_type','non_striker','match_id','player_dismissed','wicket_type'],axis = 1,inplace = True)

In [ ]:
dfcopy = df
dfcopy.columns

In [ ]:
dfcopy['target'] = 0
dfcopy

In [ ]:
dfcopy.loc[0, 'target'] = dfcopy.loc[0, 'total_runs']

dff = dfcopy
dff = dff.reset_index()
for i in range(1, len(dff)):
    if dff.loc[i,'ball'] == 0.1:
        dff.loc[i,'target'] = 0
        continue
    dff.loc[i, 'target'] = dff.loc[i-1, 'target'] + dff.loc[i, 'total_runs']

dff

In [ ]:
dfcopy = dff
dfcopy = dfcopy.reset_index()
dfcopy = dfcopy.drop(columns = ['total_runs','index','level_0'])
dfcopy

In [ ]:
dfcopy['batting_team'] = dfcopy['batting_team'].replace(['Kings XI Punjab'],'Punjab Kings')
dfcopy['batting_team'] = dfcopy['batting_team'].replace(['Delhi Daredevils'],'Delhi Capitals')
dfcopy['bowling_team'] = dfcopy['bowling_team'].replace(['Kings XI Punjab'],'Punjab Kings')
dfcopy['bowling_team'] = dfcopy['bowling_team'].replace(['Delhi Daredevils'],'Punjab Kings')
dfcopy['venue'] = dfcopy['venue'].replace(['M.Chinnaswamy Stadium'],'M Chinnaswamy Stadium')
dfcopy['venue'] = dfcopy['venue'].replace(['Wankhede Stadium, Mumbai'],'Wankhede Stadium' )
dfcopy['venue'] = dfcopy['venue'].replace(['MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium, Chepauk, Chennai'],'MA Chidambaram Stadium' )
dfcopy['venue'] = dfcopy['venue'].replace(['Punjab Cricket Association IS Bindra Stadium, Mohali','Punjab Cricket Association Stadium, Mohali'],'Punjab Cricket Association IS Bindra Stadium' )
dfcopy['venue'] = dfcopy['venue'].replace(['Rajiv Gandhi International Stadium, Uppal'],'Rajiv Gandhi International Stadium' )


In [ ]:
current_teams = ['Royal Challengers Bangalore', 'Kolkata Knight Riders' ,'Punjab Kings',
 'Chennai Super Kings', 'Rajasthan Royals', 'Mumbai Indians', 'Sunrisers Hyderabad', 'Delhi Capitals']
dfcopy = dfcopy[(dfcopy['batting_team'].isin(current_teams)) & (dfcopy['bowling_team'].isin(current_teams))]

In [ ]:
dfcopy = dfcopy.reset_index()
dfcopy = dfcopy.drop(columns = ['index'])
dfcopy

In [ ]:
current_venues = ['M Chinnaswamy Stadium','Eden Gardens', 'Feroz Shah Kotla', 'MA Chidambaram Stadium','Wankhede Stadium']

for i in range(0, len(dfcopy)):
      if dfcopy.loc[i,'venue'] not in current_venues:
             dfcopy.loc[i,'venue'] = 'Others'

In [ ]:
#ONE HOT ENCODING
from sklearn.preprocessing import LabelEncoder
import joblib

venue_encoder = LabelEncoder()
team_encoder = LabelEncoder()
striker_encoder = LabelEncoder()
bowler_encoder = LabelEncoder()
cols = ['venue','batting_team','bowling_team','striker','bowler']

dfcopy['venue'] = venue_encoder.fit_transform(dfcopy['venue'])
dfcopy['batting_team'] = team_encoder.fit_transform(dfcopy['batting_team'])
dfcopy['bowling_team'] = team_encoder.fit_transform(dfcopy['bowling_team'])
dfcopy['striker'] = striker_encoder.fit_transform(dfcopy['striker'])
dfcopy['bowler'] = bowler_encoder.fit_transform(dfcopy['bowler'])
joblib.dump(venue_encoder, 'venue_encoder.joblib')
joblib.dump(team_encoder, 'team_encoder.joblib')
joblib.dump(striker_encoder, 'striker_encoder.joblib')
joblib.dump(bowler_encoder, 'bowler_encoder.joblib')

dfcopy


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
final_df = dfcopy

In [ ]:
final_df.drop(labels='start_date', axis=True, inplace=True)

In [ ]:
#Splitting the data into train and test set
X = final_df.drop(labels='target', axis=1)
y = final_df['target'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,random_state = 42)


In [ ]:
#USING RANDOM FOREST CLASSIFIER
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()
# Train Model
forest.fit(X_train, y_train)

In [ ]:
# Evaluate Model
train_score_forest = str(forest.score(X_train, y_train)*100)
test_score_forest = str(forest.score(X_test, y_test)*100)
print(f'Train Score : {train_score_forest[:5]}%\nTest Score : {test_score_forest[:5]}%')


In [ ]:
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
print("---- Random Forest Regression - Model Evaluation ----")
print("Mean Absolute Error (MAE): {}".format(mae(y_test, forest.predict(X_test))))
print("Mean Squared Error (MSE): {}".format(mse(y_test, forest.predict(X_test))))
print("Root Mean Squared Error (RMSE): {}".format(np.sqrt(mse(y_test, forest.predict(X_test)))))

In [ ]:
joblib.dump(forest, 'rf_encoder.joblib')